Load the data from the csv file into a Pandas dataframe  
Delete any NULL addresses  
Add a new column called street which will hold the street names only (without the house number)

In [ ]:
import pandas
import re
from numpy import NaN

data = pandas.read_csv( 'data.csv', encoding= 'unicode_escape', usecols=['Address'] )
data.dropna( inplace=True)
data['Street'] = None
data

Create a road types lookup dictionary to us to make sure all road types are cleaned up with a standard name  
(road_types.lookup was created from the clean_roads Jupyter notebook)

In [ ]:
road_type_lookup = {}

with open( 'road_types.lookup', 'r') as lookup:
    for line in lookup:
        lookup, abbreviation = line.strip().split(',')
        road_type_lookup[lookup] = abbreviation

road_type_lookup



Function to extract the street name from an address
Addressess can take many forms, for example

 * 1 Smith St
 * 1A Smith St
 * Smith St
 * UNIT 1 Smith St

 It also does a lookup to the road type to make sure the road type is using a standard name and cleans up the data

 The logic goes through the address one word at a time in reverse order until it finds 

 * A word containing a number
 * A single letter word (eg A Smith St)
 * A UNIT or Lot name

 Once it finds that position it will add all the remaining words to a string and return it

In [ ]:
def get_street_name( address ):
    parts = address.split(' ')

    #  Loop each part and ignore it if its a number or contains a number
    #  or is only one character or is called UNIT or Lot
    #  Once we find one thats not we just get all the rest
    start_index = -1
    num_parts = len( parts )
    times = 0

    while times < num_parts:
        part = parts[start_index].upper()

        #  If first time then check if the road type needs to be cleaned
        #  using the road_type_lookup
        if times == 0:
            if part in road_type_lookup:
                part = road_type_lookup[part]
            parts[start_index] = part

   
        if part.isnumeric() or bool(re.search(r'\d', part)) \
           or len(part ) == 1 or part.upper() == 'UNIT' \
           or part.upper() == 'LOT':
            break
            
        start_index = start_index - 1
        times +=1 

    return ' '.join( parts[start_index+1:] )

Process all the addresses and create the street name using the get_street_name function above  
Add the street name to the 'Street' column

In [ ]:
for ind in data.index:
    address = data['Address'][ind]
    
    if not address is NaN:
        street = get_street_name( address )
        data['Street'][ind] = street

data

Query for all streets that have a length of more than 10
and count how many there are in each group

In [ ]:
result=data.query('Street.str.len() > 10')['Street'].value_counts(sort=True, ascending=False)
result

Get the street with the most number of counts

In [ ]:
print(f'The most common street name that has more than 10 letters is {result.nlargest(1).index[0]}')